### Cell 4: Time Decay and Feature Engineering

**Markdown Explanation:**

This cell defines the function `apply_time_decay`, which applies time decay to the ratings. The function adjusts the ratings based on the time since the rating was given and the release year of the movie. More recent ratings and newer movies are given higher importance. The function returns a DataFrame with adjusted ratings.

In [ ]:
def apply_time_decay(ratings_df, decay_factor, year_divisor):
    """
    Apply time decay to ratings.

    This function adjusts the ratings based on the time since the rating was given and the release year of the movie.
    More recent ratings and newer movies are given higher importance.

    Parameters:
        ratings_df (pd.DataFrame): DataFrame containing ratings data.
        decay_factor (float): Decay factor for time decay.
        year_divisor (float): Divisor for year difference to adjust importance of release year.

    Returns:
        pd.DataFrame: DataFrame with adjusted ratings.
    """
    current_time = pd.to_datetime('now')  # Get current time
    time_diff_days = (current_time - ratings_df['timestamp']).dt.total_seconds() / (3600 * 24)  # Calculate time difference in days
    time_weights = decay_factor ** (time_diff_days / 365)  # Calculate time decay weights

    year_diff = (current_time.year - ratings_df['release_year']) / year_divisor  # Calculate year difference
    adjusted_ratings = ratings_df['rating'] * time_weights / (1 + year_diff)  # Adjust ratings based on time decay and year difference

    min_rating, max_rating = adjusted_ratings.min(), adjusted_ratings.max()
    if (min_rating != max_rating) and (min_rating != float('inf')) and (max_rating != float('inf')):
        adjusted_ratings = 5.0 * (adjusted_ratings - min_rating) / (max_rating - min_rating)  # Normalize adjusted ratings to 0.5 - 5.0

    ratings_df['adjusted_rating'] = adjusted_ratings.clip(0.5, 5.0).round(1)  # Clip and round adjusted ratings to 0.5 - 5.0

    return ratings_df
